In [33]:
import json
missing_qn = []
# Replace 'your_file.json' with the path to your JSON file
with open('/home/yl3427/cylab/SOAP_MA/Output/MedicalQA/step1_final_from11.json', 'r') as file:
    data = json.load(file)

for idx, item in enumerate(data):
    try:
        question = item['Question']
    except:
        print("Error in line", idx)
        print(item['qn_num'])
        missing_qn.append(item['qn_num'])

Error in line 1
13
Error in line 4
16
Error in line 17
29
Error in line 18
30
Error in line 37
49
Error in line 46
58
Error in line 60
72
Error in line 67
79
Error in line 73
85
Error in line 75
87
Error in line 76
88
Error in line 96
108
Error in line 97
109
Error in line 98
110
Error in line 102
114
Error in line 104
116
Error in line 105
117


In [34]:
missing_qn

[13, 16, 29, 30, 49, 58, 72, 79, 85, 87, 88, 108, 109, 110, 114, 116, 117]

In [31]:
len(data)

11

In [8]:
input_json_path = "sample_data.json"
f"{input_json_path.split('.')[0]}_with_baseline.json"


'sample_data_with_baseline.json'

In [4]:
import pandas as pd
df = pd.read_csv("/home/yl3427/cylab/llm_reasoning/reasoning/data/step3_ALL.csv")
len(df)

137

In [4]:
input_json_path.split(".")

['sample_data', 'json']

In [ ]:
# merdge two json files

import json
import copy
import os

def deep_merge(dict1, dict2):
    """
    Recursively merge dict2 into dict1.
    - If a key exists in both dicts and both values are themselves dicts, 
      recurse into those.
    - Otherwise, dict2’s value overwrites dict1’s value.
    """
    for key, value in dict2.items():
        if key not in dict1:
            dict1[key] = value
        else:
            if isinstance(dict1[key], dict) and isinstance(value, dict):
                deep_merge(dict1[key], value)
            else:
                # Overwrite dict1's value with dict2's value
                dict1[key] = value
    return dict1

def merge_json_files(file1, file2, output_file):
    # 1) Load both JSON files
    with open(file1, "r", encoding="utf-8") as f:
        data1 = json.load(f)  # List of dicts
    with open(file2, "r", encoding="utf-8") as f:
        data2 = json.load(f)  # List of dicts

    # 2) Convert the first list into a dictionary keyed by "qn_num"
    merged_dict = {}
    for item in data1:
        qn_num = item["qn_num"]
        merged_dict[qn_num] = copy.deepcopy(item)  # Use copy if you want to avoid mutating the original

    # 3) For each item in data2, either add or merge
    for item in data2:
        qn_num = item["qn_num"]
        if qn_num not in merged_dict:
            merged_dict[qn_num] = copy.deepcopy(item)
        else:
            # deep-merge fields from item into existing
            deep_merge(merged_dict[qn_num], item)

    # 4) Convert the merged dictionary back to a list
    final_list = list(merged_dict.values())
    print(f"Merged {len(data1)} items from file1 and {len(data2)} items from file2")
    print(f"Final list contains {len(final_list)} items")

    # 5) Write to the output file
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(final_list, f, indent=2, ensure_ascii=False)

if __name__ == "__main__":
    # Example usage:
    file1 = "/home/yl3427/cylab/SOAP_MA/Output/MedicalQA/step1_merged_missing.json"
    file2 = "/home/yl3427/cylab/SOAP_MA/Output/MedicalQA/step1_final_missing.json"
    output_file = "/home/yl3427/cylab/SOAP_MA/Output/MedicalQA/step1_merged.json"

    merge_json_files(file1, file2, output_file)
    print(f"Merged file saved as: {output_file}")


Merged 11 items from file1 and 108 items from file2
Final list contains 119 items
Merged file saved as: /home/yl3427/cylab/SOAP_MA/Output/MedicalQA/step1_merged_missing.json
